# Easy as Py

## Summary

### Table of Contents

[Functions](#functions)

[Google Cloud API](#google-cloud-api)

[Spoonacular API](#spoonacular-api)

### Dependencies

#### Installations

In [1]:
# FuzzyWuzzy
# \!pip install fuzzywuzzy

# Docx
# \!pip install python-docx

# Google API client library
# \!pip install --upgrade google-api-python-client oauth2client # google-auth-httplib2 google-auth-oauthlib

# GSpread
# \!pip install gspread

#### Imports

In [45]:
from pathlib import Path
import csv
import pandas as pd
# import panel as pn
# pn.extension('plotly')
# import plotly.express as px
# import plotly.io as pio
# pio.renderers.default = 'iframe_connected'  # Bypass mimetype 'renderer not found'
# import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns

import sys
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pathlib import Path
from dotenv import load_dotenv

import json, requests
from pandas.io.json import json_normalize

# Docx
import docx
from docx import Document
from docx.shared import Inches
from os import listdir
from os.path import isfile, join

# Google Sheets API
from pprint import pprint
from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials

# GSpread by burnash
import gspread

# 'chatter` functions from chatter.py

# from chatter import *

### Tokens, Scopes, and Credentials

In [3]:
# API Keys for Spoonacular and Google Cloud
load_dotenv('./tokens/token.env')
spoon_key = os.getenv('SPOONACULAR_KEY')
gcloud_key = os.getenv('GCLOUD_KEY')
gcloud_oauth_key = os.getenv('GCLOUD_OAUTH_KEY')
gcloud_oauth_secret = os.getenv('GCLOUD_OAUTH_SECRET')

# Endpoints for Spoonacular
# Requests should be formatted as "spoon_url + endpoint + '?query=' + query_params + key_url"
spoon_url = 'https://api.spoonacular.com'
key_url = '&apiKey=' + spoon_key
# Ingredient search endpoint
search_ing_url = spoon_url + '/food/ingredients/search?query='

# Scopes for Google Cloud
scope = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/documents'
    'https://spreadsheets.google.com/feeds',
]

# Add account credentials for Google Cloud
creds = ServiceAccountCredentials.from_json_keyfile_name('./tokens/easy-as-py-service-account.json')

# Authorize clientsheet for Google Cloud
client = gspread.authorize(creds)

## Functions

### Docx Functions

In [4]:
# Path to recipes directory
recipes_dir = './recipes/'

# Get all recipe file names from directory as a list
def get_recipe_list():
    global recipe_files
    recipe_files = [f for f in listdir(recipes_dir) if isfile(join(recipes_dir, f))]
    return recipe_files

# Get nest list of ingredients for a recipe from its docx
def get_recipe_from_docx(docx_file):
    docx_text = docx.Document(f"{recipes_dir}{docx_file}")
    data = {}
    paragraphs = docx_text.paragraphs
    for i in range(2, len(docx_text.paragraphs)):
        data[i] = docx_text.paragraphs[i].text.split('\t')
    data_values = list(data.values())
    return data_values

# To verify data, print the ingredients of all recipes as nested lists
def print_all_recipes_ingredients():
    for i in recipe_files:
        print(i)
        print(get_recipe_from_docx(f"{i}"), '\n')

# Create a DataFrame from a recipe's data
def create_df_from_recipe(recipe_data):
    df = pd.DataFrame(recipe_data, columns=['Ingredient_Name','Recipe_Qty']).dropna()
    df = df.join(df['Recipe_Qty'].str.split(' ', 1, expand=True)).drop(columns=['Recipe_Qty']).rename(columns={0:'Recipe_Qty',1:'Recipe_Unit'})
    return df

### Google Sheets Functions

In [5]:
# Get instance of spreadsheet using its title
def open_spreadsheet(spreadsheet_title):
    return client.open(spreadsheet_title)

# Nested function to get specific sheet (tab) from spreadsheet instance
def open_sheet(spreadsheet_title, tab_index):
    sheet = open_spreadsheet(spreadsheet_title).get_worksheet(tab_index)
    return sheet

# Get all sheet data
def get_sheet_data(spreadsheet_title, tab_index):
    sheet = open_sheet(spreadsheet_title, tab_index)
    sheet_data = sheet.get_all_records()
    return sheet_data

# Convert a letter into its numerical equivalent for use in `get_cell()`
# Columns in spreadsheets use letters.
# However, the API calls them via their numerical placement.
# `ord()` returns an integer that represents a Unicode character,
# which differs between lower and upper cases.
def letter_to_num(column_letter):
    number = ord(column_letter.lower()) - 96
    return number

# Get data from specified cell
def get_cell(sheet_var, column_letter, row_num):
    column_number = letter_to_num(column_letter)
    return sheet.cell(col=column_number,row=row_num)

# Turn sheet dictionary into a dataframe
def create_df_from_sheet(sheet_data):
    df = pd.DataFrame.from_dict(sheet_data)
    return df

# Create df for specified tab in 'Order Sheet' with selected columns
# Keep rows where the 'Best Price' IS NOT equal to 0
def create_order_sheet_df(tab_index):
    df = create_df_from_sheet(
        get_sheet_data(
            'Order Sheet', tab_index
        )
    )[['Item','Order Unit','Best Price']]
    df = df[df['Best Price'] != 0]
    df = df.rename(columns={
        'Item':'Ingredient_Name',
        'Order Unit':'Purchase_Unit',
        'Best Price':'Purchase_Price_$'
    })
    return df

# Clean menu engineering DataFrame by dropping rows with no value in 'Item_Name'
def clean_engineering_df(df):
    df.drop(df[df['Item_Name'] == ''].index, inplace=True)
    return df

### Spoonacular Functions

In [70]:
# Function that takes in an ingredient name and returns up to 20 matches
# Example request: https://api.spoonacular.com/food/ingredients/search?query=banana&number=2&sort=calories&sortDirection=desc
def search_ing(ing):
    return requests.get(search_ing_url + ing + '&number=20' + key_url).json()

# Get ingredient `id`
def get_ing_id(ing):
    ingredient = search_ing(ing)['results'][0]
    print(f"Match for {ing}:")
    print(ingredient)

    ing_id = ingredient['id']
    return ing_id

# Get ingredient information
def get_ing_info(ing, ing_amount, unit):
        ing_id = get_ing_id(ing)
        return requests.get(f"{spoon_url}/food/ingredients/{ing_id}/information?amount={ing_amount}&unit={unit}{key_url}").json()
    
# Create df from ingredient info
def ing_info_to_df(ing, ing_amount, unit):
    ing_info = get_ing_info(ing, ing_amount, unit)
    df = pd.json_normalize(ing_info)[['id','name','amount','unit','estimatedCost.value']]
    df['estimatedCost.value'] = (df['estimatedCost.value'] / 100).round(2)
    df.columns = [['ID','Ingredient_Name','Recipe_Qty','Recipe_Unit','Recipe_Cost_Per_Unit_$']]
    return df

# Script Flow

## Read Data from Original Sources into Python

### Get Recipe Ingredients from Docx Files

In [7]:
# Get list of all recipes from directory
recipe_list = get_recipe_list()
recipe_list

['basil_ricotta_recipe.docx',
 'cheesecake_recipe.docx',
 'chicken_parm_meatballs_recipe.docx',
 'marinara_recipe.docx']

In [8]:
# Print the ingredients of all recipes
# print_all_recipes_ingredients()

In [9]:
# Get a recipe's list of ingredients and quantities
data = get_recipe_from_docx(recipe_list[0])

In [10]:
# Create a DataFrame of the first recipe 'Basil Ricotta'
basil_ricotta_df = create_df_from_recipe(data)
basil_ricotta_df

,Ingredient_Name,Recipe_Qty,Recipe_Unit
0,Ricotta,2,lbs
1,Fresh basil,1,c
2,Lemon (juice and zest),1,ea
3,Ground nutmeg,1,tsp
4,Tri-mix,2,tsp
5,EVOO,⅛,c
6,Heavy cream,¼,c


### Get Ingredient Purchase Prices from Google Sheets

In [11]:
# Create DataFrames from tabs in spreadsheet
order_sheet_df = pd.DataFrame()
produce_df = create_order_sheet_df(0)
groceries_df = create_order_sheet_df(1)
poultry_seafood_df = create_order_sheet_df(2)
spices_df = create_order_sheet_df(4)

In [12]:
produce_df

,Ingredient_Name,Purchase_Unit,Purchase_Price_$
0,"Asparagus, 11lb",cse/11ct,23.97
1,Arugula,3 lb cse,18.36
2,"Apples,granny smith",ea,0.89
3,"Avocado, Hass, Ripe",ea,1.60
4,Bananas ( 3x per week ),lb,0.67
...,...,...,...
75,"Tomatoes, 6x6",cs,46.86
76,"Tomatoes, Grape",cs/12/ea,20.86
77,"Tomatoes, Plum",cs,38.31
78,"Tomatoes, Tomatillo",lb,2.58


In [13]:
# Combine all DataFrames into one
order_sheet_df = order_sheet_df.append([produce_df,groceries_df,poultry_seafood_df,spices_df])
order_sheet_df

,Ingredient_Name,Purchase_Unit,Purchase_Price_$
0,"Asparagus, 11lb",cse/11ct,23.97
1,Arugula,3 lb cse,18.36
2,"Apples,granny smith",ea,0.89
3,"Avocado, Hass, Ripe",ea,1.6
4,Bananas ( 3x per week ),lb,0.67
...,...,...,...
66,Tri Mix,50lb,120.0
67,Tumerick,18oz,6.75
68,Vanilla Beans,4oz,22.95
69,Wasabi,4lb,36.5


### Calculate Individual Recipe Costs

#### Using Order Sheet

In [14]:
recipe_list

['basil_ricotta_recipe.docx',
 'cheesecake_recipe.docx',
 'chicken_parm_meatballs_recipe.docx',
 'marinara_recipe.docx']

In [75]:
cheesecake_df = create_df_from_recipe(get_recipe_from_docx(recipe_list[1]))
cheesecake_df

,Ingredient_Name,Recipe_Qty,Recipe_Unit
0,Cream cheese,20,oz
1,Granulated sugar,17.5,oz
2,Sour cream,4,oz
3,All purpose flour,3,Tbsp
4,Eggs,5,each
5,Egg yolks,2,each
6,Vanilla extract,3,tsp
7,Graham cracker crumbs,6.75,oz
8,Unsalted butter,6,oz


In [46]:
def price_match(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2.iloc[key2].tolist()
    m = df_1.iloc[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [68]:
mat1 = []
mat2 = []
list1 = cheesecake_df['Ingredient_Name'].tolist()
list2 = order_sheet_df['Ingredient_Name'].tolist()
print(list1)
print(list2[:10])
threshold = 90

for i in list1:
    mat1.append(process.extract(i, list2, limit=2))
pd.DataFrame(mat1)

['Cream cheese', 'Granulated sugar', 'Sour cream', 'All purpose flour', 'Eggs', 'Egg yolks', 'Vanilla extract', 'Graham cracker crumbs', 'Unsalted butter']
['Asparagus, 11lb', 'Arugula', 'Apples,granny smith', 'Avocado, Hass, Ripe', 'Bananas   ( 3x per week ) ', 'Basil', 'Beets, Purple, and Golden', 'Blueberries, Fresh', 'Bok Choy, Baby', 'Bok Choy, Large']


,0,1
0,"(Cheese,Cream, 95)","(Cheese, Mozz, Stick's - Sysco Imp, 86)"
1,"(Sugar,granulated, 95)","(Garlic, Granulated, 81)"
2,"(Cream, Sour, 95)","(Cream, Culinary, Minors, 86)"
3,"(Flour Sir Lancelott High Gluten (pizza), 86)","(Flour, PATENT 4X, Pillsbury, 86)"
4,"(Eggs bag break-o-morn, 90)","(Eggs, Large, Grade A, 90)"
5,"(Eggs, Yolks, Frozen, 86)","(Leeks, 68)"
6,"(Vanilla extract, Imitation, 90)","(Ice Cream, Vanilla,B&J (#3476), 86)"
7,"(Crackers, Graham, Crushed, 82)","(Rib Rub , 64)"
8,"(Butter, Unsalted, 95)","(Peanut Butter, Skippy,Smooth, 86)"


#### Using Spoonacular

In [91]:
cheesecake_df.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [84]:
# cheesecake_df['Ingredient_Name']#,cheesecake_df[1],cheesecake_df[2]
# cheesecake_df['Ingredient_Name'][0]
ing_info_to_df(cheesecake_df.iloc[0][0],cheesecake_df.iloc[0][1],cheesecake_df.iloc[0][2])

Match for Cream cheese:
{'id': 1001017, 'name': 'cheese soft cream', 'image': 'cream-cheese.jpg'}


,ID,Ingredient_Name,Recipe_Qty,Recipe_Unit,Recipe_Cost_Per_Unit_$
0,1001017,cheese soft cream,20.0,oz,6.07


### Write Recipe Costs into Menu Engineering GSheet

### Visualize Menu Engineering Data

In [15]:
# import turtle
# turtle.write('Congratulations, FinTech Bootcamp Class 2021!')